In [99]:
import praw
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
# get reddit secret & script i've saved as .txt in the working directory

with open('reddit_oa_script.txt') as f:
    
    reddit_script = f.read().strip()
    
with open('reddit_oa_secret.txt') as f:
    
    reddit_secret = f.read().strip()
    
with open('reddit_id.txt') as f:
    
    pw = f.read().strip()

In [101]:
# user_agent is the name of the app

reddit  = praw.Reddit(client_id=reddit_script,
                          client_secret=reddit_secret,
                          user_agent='get_corpora',
                          username='neurodivergent_ai',
                          password=pw)


In [102]:
# get the subreddit

subreddit_1 = reddit.subreddit('The_Donald')

# get the top (upvoted) 1000 threads

subreddit_1_top = subreddit_1.top(limit=1000)

In [103]:
# get the top 5 submissions to check

for post in subreddit_1.top(limit=5):
    
    print(post.title, '\n')

DONALD J. TRUMP DECLARED THE WINNER! 

I’m Donald J. Trump and I'm Your Next President of the United States. 

The New Algorithm Was Designed To Keep President Trump From The Front Page... But Sadly, That Won't Happen. 

CNN will soon be #1...when searching for the term "Fake News". Congratulations to CNN on this major achievement. 

ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A. IF TRUMP WINS, WE NOW DEMAND YOU TO FOLLOW THROUGH!!!! OUT! OUT! OUT! 



In [104]:
# build a dataset

# choose information to gather

columns_dict = {'title':[],
               'id':[],
               'url':[],
               'date':[],
               'score':[],
               'num_comments':[],
               'body':[]}

# build a dictionary

for post in subreddit_1_top:
    
    columns_dict['title'].append(post.title)
    
    columns_dict['id'].append(post.id)
    
    columns_dict['url'].append(post.url)
    
    columns_dict['date'].append(post.created)
    
    columns_dict['score'].append(post.score)
    
    columns_dict['num_comments'].append(post.num_comments)
    
    columns_dict['body'].append(post.selftext)

# convert dictionary to a dataframe

subreddit_1_df = pd.DataFrame(columns_dict)

In [105]:
subreddit_1_df.head()

,title,id,url,date,score,num_comments,body
0,DONALD J. TRUMP DECLARED THE WINNER!,5bzjv5,https://www.reddit.com/r/The_Donald/comments/5...,1.478706e+09,66712,12097,#Meme Magic is REAL.\n\n#Donald J. Trump is th...
1,I’m Donald J. Trump and I'm Your Next Presiden...,4uxdbn,https://www.reddit.com/r/The_Donald/comments/4...,1.469687e+09,42120,20807,Hello The_Donald readers and the entire Reddit...
2,The New Algorithm Was Designed To Keep Preside...,5gvy1j,https://i.sli.mg/gu0oHA.png,1.481092e+09,40969,835,
3,CNN will soon be #1...when searching for the t...,5jt9xs,https://i.redd.it/7jeyhzgku65y.png,1.482472e+09,37276,724,
4,ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A....,5bz5ds,https://www.reddit.com/r/The_Donald/comments/5...,1.478702e+09,36615,1685,Now that we have proven our commitment to the ...


In [106]:
# create a copy to work on

reddit_df = subreddit_1_df.copy()

In [107]:
# grab texts to build into corpus

titles_df = reddit_df['title']

posts_df = reddit_df['body']

In [108]:
titles_df.head(10)

0                 DONALD J. TRUMP DECLARED THE WINNER!
1    I’m Donald J. Trump and I'm Your Next Presiden...
2    The New Algorithm Was Designed To Keep Preside...
3    CNN will soon be #1...when searching for the t...
4    ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A....
5                             IMMINENT VICTORY THREAD.
6    Hey admins, we found a picture of your wife's ...
7             Oh yeah, nothing fucky going on here :^)
8                     Can't Stump your President Trump
9    It's official: Trump will become the first U.S...
Name: title, dtype: object

In [109]:
posts_df.head(10)

0    #Meme Magic is REAL.\n\n#Donald J. Trump is th...
1    Hello The_Donald readers and the entire Reddit...
2                                                     
3                                                     
4    Now that we have proven our commitment to the ...
5    #THIS IS NOT A DRILL. WE ARE ACTUALLY GOING TO...
6                                                     
7                                                     
8                                                     
9                                                     
Name: body, dtype: object

In [110]:
# experiment
# need to toeknize, etc

text = posts_df[4].split()

print(text)

['Now', 'that', 'we', 'have', 'proven', 'our', 'commitment', 'to', 'the', 'cause,', 'now', 'that', 'we', 'have', 'voiced', 'and', 'acted', 'on', 'our', 'patriotism,', 'now', 'that', 'against', 'all', 'odds', 'and', 'constant', 'criticism', 'with', 'the', 'world', 'against', 'us,', 'we', 'have', 'won,', 'and', 'we', 'call', 'for', 'this:', '#All', 'of', 'you', 'elitist', 'hollywood', 'celebrities', 'who', 'vowed', 'to', 'betray', 'your', 'country', 'and', 'leave,', 'to', 'abandon', 'and', 'shun', 'those', 'who', 'you', 'spent', 'your', 'careers', 'milking', 'for', 'your', 'posh', 'lifestyles,', 'we', 'ask,', 'nay,', 'demand', 'that', 'you', 'follow', 'through', 'on', 'your', 'claims', 'to', 'leave', 'the', 'country.', 'We', 'as', 'Americans', 'have', 'spoken,', 'and', 'you', 'brushed', 'us', 'and', 'our', 'movement', 'off,', 'as', 'if', 'you', 'were', 'somehow', 'above', 'us,', 'forgetting', 'that', 'it', 'was', 'us', 'who', 'made', 'you.', '**FUCK', 'YOU!**', 'You', 'think', 'you', 'ar

the text needs to be cleaned. lots of stopwords, punctuation, & non-alpha characters. case needs to be lowered etc.

In [111]:
def clean_text(doc):
    '''
    tokenizes input text, converts to lowercase,
    removes punctuation/non-alphabetic characters
    and stopwords. returns clean document.
    
    '''
    
    # tokenize
    
    split_tokens = doc.split()
    
    # grab punctuation list
    
    punctuation = string.punctuation
    
    # set up regex 
    
    punct_filter = re.compile('[%s]' % re.escape(punctuation))
    
    # filter out punctuation
    
    no_punct_tokens = [punct_filter.sub('', char) for char in split_tokens]
    
    # remove non-alphabetic tokens
    
    alpha_tokens = [t for t in no_punct_tokens if t.isalpha()]
    
    # lowercase
    
    lower_tokens = [t.lower() for t in alpha_tokens]
    
    # get stop words
    
    stop_words = set(stopwords.words('english'))
    
    # filter out stopwords
    
    go_tokens = [t for t in lower_tokens if not t in stop_words]
    
    # remove very short tokens
    
    clean_tokens = [word for word in go_tokens if len(word) > 1]
    
    return clean_tokens
    

In [112]:
# test

raw_text = posts_df[4]

clean_text(raw_text)

['proven',
 'commitment',
 'cause',
 'voiced',
 'acted',
 'patriotism',
 'odds',
 'constant',
 'criticism',
 'world',
 'us',
 'call',
 'elitist',
 'hollywood',
 'celebrities',
 'vowed',
 'betray',
 'country',
 'leave',
 'abandon',
 'shun',
 'spent',
 'careers',
 'milking',
 'posh',
 'lifestyles',
 'ask',
 'nay',
 'demand',
 'follow',
 'claims',
 'leave',
 'country',
 'americans',
 'spoken',
 'brushed',
 'us',
 'movement',
 'somehow',
 'us',
 'forgetting',
 'us',
 'made',
 'fuck',
 'think',
 'good',
 'us',
 'well',
 'good',
 'follow',
 'vapid',
 'empty',
 'threat',
 'mistakenly',
 'pridefully',
 'consider',
 'get',
 'fuck',
 'fame',
 'celebrity',
 'status',
 'despicable',
 'shall',
 'say',
 'deplorable',
 'better',
 'us',
 'important',
 'us',
 'would',
 'nothing',
 'without',
 'us',
 'dare',
 'pretend',
 'us',
 'list',
 'demand',
 'leave',
 'asap',
 'amy',
 'unfunny',
 'schumer',
 'lena',
 'dunham',
 'barbra',
 'streisand',
 'bryan',
 'cranston',
 'miley',
 'cyrus',
 'amy',
 'schumer',


In [118]:
def clean_all(titles, posts):
    '''
    takes 2 input lists of documents (designed for
    titles & posts). iteratively applies
    clean_text() function, appends all texts 
    to single list. returns 2 lists of 
    clean texts.
    
    '''
    
    clean_post_list = []
    
    clean_title_list = []

    # clean posts

    for post in posts:
    
        post = clean_text(post)
    
        clean_post_list.append(post)
    
    # clean titles

    for title in titles:
    
        title = clean_text(title)
    
        clean_title_list.append(title)
    
    return clean_post_list, clean_title_list

In [119]:
clean_posts, clean_texts = clean_all(titles_df, posts_df)

In [120]:
print(clean_posts[4], '\n')
print(clean_titles[4])



['proven', 'commitment', 'cause', 'voiced', 'acted', 'patriotism', 'odds', 'constant', 'criticism', 'world', 'us', 'call', 'elitist', 'hollywood', 'celebrities', 'vowed', 'betray', 'country', 'leave', 'abandon', 'shun', 'spent', 'careers', 'milking', 'posh', 'lifestyles', 'ask', 'nay', 'demand', 'follow', 'claims', 'leave', 'country', 'americans', 'spoken', 'brushed', 'us', 'movement', 'somehow', 'us', 'forgetting', 'us', 'made', 'fuck', 'think', 'good', 'us', 'well', 'good', 'follow', 'vapid', 'empty', 'threat', 'mistakenly', 'pridefully', 'consider', 'get', 'fuck', 'fame', 'celebrity', 'status', 'despicable', 'shall', 'say', 'deplorable', 'better', 'us', 'important', 'us', 'would', 'nothing', 'without', 'us', 'dare', 'pretend', 'us', 'list', 'demand', 'leave', 'asap', 'amy', 'unfunny', 'schumer', 'lena', 'dunham', 'barbra', 'streisand', 'bryan', 'cranston', 'miley', 'cyrus', 'amy', 'schumer', 'jon', 'stewart', 'cher', 'chelsea', 'handler', 'samuel', 'jackson', 'whoopi', 'goldberg', '